In [14]:
from neo4j import GraphDatabase

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j+s://0091e6ed.databases.neo4j.io"
AUTH = ("neo4j", "lOyQFg_OtF978tResvL0CKmXMm0APXm0-v5PqzAjAVQ")

driver = GraphDatabase.driver(URI, auth=AUTH)
driver.verify_connectivity()

    

In [15]:
session = driver.session() 
result = session.run("MATCH (n) RETURN n LIMIT 5")
for record in result:
    print(record["n"])

<Node element_id='4:6cdce4f9-ea4d-44e6-95eb-19e1317a3f68:0' labels=frozenset({'Country', 'Place'}) properties={'code': 'PL', 'name': 'Poland'}>
<Node element_id='4:6cdce4f9-ea4d-44e6-95eb-19e1317a3f68:1' labels=frozenset({'Country', 'Place'}) properties={'code': 'US', 'name': 'United States'}>
<Node element_id='4:6cdce4f9-ea4d-44e6-95eb-19e1317a3f68:2' labels=frozenset({'Country', 'Place'}) properties={'code': 'ES', 'name': 'Spain'}>
<Node element_id='4:6cdce4f9-ea4d-44e6-95eb-19e1317a3f68:3' labels=frozenset({'Country', 'Place'}) properties={'code': 'FR', 'name': 'France'}>
<Node element_id='4:6cdce4f9-ea4d-44e6-95eb-19e1317a3f68:4' labels=frozenset({'Country', 'Place'}) properties={'code': 'IT', 'name': 'Italy'}>


In [ ]:
def get_schema(tx):
            schema_data = {}
            query = f"""
                MATCH (table_node:{table_label})
                OPTIONAL MATCH (table_node)-[:{relationship_type}]->(attribute_node:{attribute_label})
                RETURN table_node.{table_name_property} AS tableName,
                       collect(attribute_node.{attribute_name_property}) AS attributes
            """
            result = tx.run(query)
            for record in result:
                table_name = record["tableName"]
                attributes = record["attributes"] if record["attributes"] else [] # Handle cases with no attributes
                schema_data[table_name] = attributes
            return schema_data

        with driver.session() as session:
            schema = session.read_transaction(get_schema)
            return schema

In [17]:
# Analyze node labels and properties to identify table structures
label_properties = {}
relationship_types = set()

# Analyze nodes
for node in nodes_data:
    labels = node['labels']
    for label in labels:
        if label not in label_properties:
            label_properties[label] = set()
        # Add all properties for this node type
        for prop in node.keys():
            if prop != 'labels':
                label_properties[label].add(prop)

# Analyze relationships
result = session.run(relationships_query)
for record in result:
    relationship_types.add(record['type'])

# Print table structures
print("Node Tables:")
for label, props in label_properties.items():
    print(f"\nTable: {label}")
    print("Columns:", ", ".join(['id'] + list(props)))

print("\nRelationship Tables:")
for rel_type in relationship_types:
    print(f"\nTable: {rel_type}")
    print("Columns: id, source_id, target_id, properties")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 4, column: 5, offset: 73} for query: '\nMATCH (a)-[r]->(b)\nRETURN type(r) as type, properties(r) as props, \n    id(a) as source_id, properties(a) as source_props,\n    id(b) as target_id, properties(b) as target_props\n'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 5, column: 5, offset: 128} for query: '\nMATCH (a)-[r]->(b)\nRETURN type(r) as type, properties(r) as props, \n    id(a) as source_id, properties(a) as source_props,\n    

Node Tables:

Table: Place
Columns: id, name, code, coordinates

Table: Country
Columns: id, name, code

Table: City
Columns: id, name, code, coordinates

Table: Sustenance
Columns: id, lat, housenumber, cuisine, city, name, street, lon, website

Table: Restaurant
Columns: id, lat, housenumber, cuisine, city, name, street, lon, website

Table: PlaceToSleep
Columns: id, lat, housenumber, city, name, street, lon, website

Table: Hotel
Columns: id, lat, housenumber, city, name, street, lon, website

Table: Person
Columns: id, name, age, blog_address

Table: Trip
Columns: id, duration, year_season, type, name

Relationship Tables:

Table: STARTS_FROM
Columns: id, source_id, target_id, properties

Table: IS_LOCATED_IN
Columns: id, source_id, target_id, properties

Table: BELONGS_TO
Columns: id, source_id, target_id, properties

Table: WENT_FOR
Columns: id, source_id, target_id, properties

Table: TO
Columns: id, source_id, target_id, properties

Table: LIVES_IN
Columns: id, source_id, targe

In [9]:
session.close()
driver.close()